In [1]:
import pandas as pd
df = pd.read_csv("Crude_Oil_Data.csv")

In [97]:
import numpy as np

df.columns
df.duplicated().sum()
df.isna().sum()
df['Date'] = df['Date'].apply(lambda x : x.split(" ")[0])
df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df.drop(columns="Date",inplace=True)

# Cyclical Encoding for Month
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

x = df.drop(columns=["High","Low"])
y = df[["High","Low"]]

# ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Year', 'Month','DayOfWeek', 'Month_sin', 'Month_cos']

In [98]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler_feature = StandardScaler()
x_scaled = scaler_feature.fit_transform(x)
scaler_target = StandardScaler()
y_scaled = scaler_target.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x_scaled,y_scaled,test_size=0.3)

In [99]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

model = MultiOutputRegressor(LinearRegression())
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
score = model.score(x_test,y_test)
print("SCORE : ",score)

mse_l = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print("MSE :", mse_l)
r2_l = r2_score(y_test, y_pred, multioutput='raw_values')
print("R2 SCORE : ",r2_l)


SCORE :  0.9983528500881356
MSE : [0.00152795 0.00168191]
R2 SCORE :  [0.99842323 0.99828247]


In [100]:
import joblib 

model_package = {
    'scaler_feature': scaler_feature,
    'scaler_target': scaler_target,
    'model': model
}

joblib.dump(model_package, 'model_with_scalers.pkl')

['model_with_scalers.pkl']